In [1]:
from langchain.vectorstores import FAISS

from langchain.schema import Document
from langchain.vectorstores import Chroma

## Text Splitting & Docloader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.storage import InMemoryStore
from langchain.document_loaders import TextLoader

from langchain_community.embeddings import OllamaEmbeddings

In [3]:
loaders = [
    TextLoader('data/edital_text/edital_text.txt')
]
docs = []
for l in loaders:
    docs.extend(l.load())

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=50)
texts = text_splitter.split_documents(docs)

In [4]:
# Helper function for printing docs

def pretty_print_docs(docs):
    print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))

In [5]:
embeddings = OllamaEmbeddings(model="mistral-nemo:latest", temperature=0.5)

In [9]:
retriever = Chroma.from_documents(texts,
                                 embeddings
                                 ).as_retriever()

docs = retriever.get_relevant_documents("Como funciona o sistema de cotas na UFPR?")
#lets look at the docs
pretty_print_docs(docs)

/tmp/ipykernel_196458/2523161585.py:5: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents("Como funciona o sistema de cotas na UFPR?")


Document 1:

por cento) de cada grupo alocados no turno vespertino. 
 
9.8 O Anexo XVII  contém outras informações sobre o turno de funcionamento dos demais cursos. 
 
9.9 Serão emitidos, com base no desempenho dos candidatos: 
 
a) Um relatório básico para divulgação, organizado por curso e turno, em ordem alfabética, contendo 
os nomes dos candidatos classificados para a chamada geral, sem menção de classificação ou 
de opção por categoria de concorrência; 
b) Um relatório básico para divulgação, em ordem decrescente de desempenho, organ izado por 
curso, turno e modalidade de concorrência até o limite de vagas previstas para cada um a delas, 
contendo os nomes dos candidatos classificados para a chamada geral; 
 
9.10 Caberá ao Reitor homologar os resultados do PS-UFPR e divulgar a lista dos c lassificados por vaga 
nos cursos. 
 
9.11 Somente serão consideradas oficiais e válidas, para todos os efeitos, as listas dos classificados por 
vaga nos cursos divulgadas no site  do NC/UFPR

In [13]:
from langchain_community.chat_models import ChatOllama
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

# making the compressor
llm = ChatOllama(model="mistral-nemo:latest",
                 max_tokens=206,  
                 temperature=0.3,
                 top_k=75,
                 top_p=0.2)
compressor = LLMChainExtractor.from_llm(llm)

# it needs a base retriever (we're using FAISS Retriever) and a compressor (Made above)
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor,
                                                       base_retriever=retriever)

In [16]:
compressor.llm_chain.get_prompts()

[PromptTemplate(input_variables=['context', 'question'], input_types={}, output_parser=NoOutputParser(), partial_variables={}, template='Given the following question and context, extract any part of the context *AS IS* that is relevant to answer the question. If none of the context is relevant return NO_OUTPUT. \n\nRemember, *DO NOT* edit the extracted parts of the context.\n\n> Question: {question}\n> Context:\n>>>\n{context}\n>>>\nExtracted relevant parts:')]

In [17]:
compressed_docs = compression_retriever.get_relevant_documents("Como funcionam as cotas?")
pretty_print_docs(compressed_docs)

Document 1:

"3.5.3 A solicitação de alteração somente poderá ser realizada após o pagamento da taxa de inscrição a que se refere o item 3.2.3 deste edital, ou após a homologação da isenção dessa taxa."
